# Multi-Sensor Atmospheric Correction in Google Earth Engine

**Description:** This script allows to do atmospheric correction for list of images of Sentinel-2 and Landsat sensors, especifically for images over coastal or oceanic areas. These settings can be modified from the *parameters.py* module to work with images over inland areas (See line 36 in that module). The script does AC automatically, the user only needs to provide the right **mission**, **imageID**, and specific **assetID** to export the processed image to your EE Assets.<br/>
More sensors can be added by modifying the *mission_specifics.py* and *parameters.py* modules to properly work with the available collections in GEE and [Py6S](https://github.com/robintw/Py6S/blob/master/Py6S/Params/wavelength.py).<br/>

Script modified from https://github.com/samsammurphy/gee-atmcorr-S2<br/>
By Luis Lizcano-Sandoval<br/>
College of Marine Science, University of South Florida<br/>
10/14/2020<br/>

### Import modules and initialize Earth Engine



In [ ]:
import ee
from Py6S import *
import datetime
import math
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
from atmospheric import Atmospheric
import mission_specifics as mn
from parameters import BOA
import timeit

ee.Initialize()

### Earth Engine Collections
Set the collection of interest using one this specific categories:


In [ ]:
#mission = 'Sentinel2'
mission = 'Landsat8'
#mission = 'Landsat7'
#mission = 'Landsat5'

### Image ID's
Paste the list of image IDs.

In [ ]:
# imageID = ['20190203T160459_20190203T160859_T17RMH',
#  '20190228T160211_20190228T160836_T17RMH',
#  '20191125T160611_20191125T161352_T17RMH',
#  '20191105T160441_20191105T161109_T17RMH',
#  '20190208T160421_20190208T161051_T17RMH',
#  '20191130T160619_20191130T160655_T17RMH',
#  '20190302T160509_20190302T160509_T17RMH',
#  '20191210T160639_20191210T160735_T17RMH',
#  '20191215T160651_20191215T161517_T17RMH',
#  '20191016T160241_20191016T161055_T17RMH',
#  '20190215T160511_20190215T160508_T17RMH',
#  '20190411T160519_20190411T160516_T17RMH',
#  '20190129T160521_20190129T161214_T17RMH',
#  '20191110T160459_20191110T160501_T17RMH'] #Sentinel-2
imageID = ['LC08_017040_20191013', 'LC08_017040_20191130'] #Landsat8
#imageID = 'LE07_006038_20020315' #Landsat7
#imageID = 'LT05_017040_20061009' #Landsat5

## Sort list
imageID = sorted(imageID)
print(sorted(imageID))

### Load Collection
Get the respective scenes from the collection

In [ ]:
# Load collection
collection = ee.ImageCollection(mn.eeCollection(mission)).filter(ee.Filter.inList('system:index',imageID))
firstImage = collection.first()
count = collection.size()
print('Number of images in this collection: ', count.getInfo())

#  ImageCollection to List           
col_list = collection.toList(count) #ee.List
#col_size = col_list.size().getInfo() #Python list
#print(col_size)

# Select an image for debugging
for i in range(count.getInfo()):
    image = col_list.get(i)#first()
    print('Image '+str(i)+':', image.getInfo()['properties']['system:index'])

# Date
dateString = datetime.datetime.utcfromtimestamp(firstImage.get('system:time_start').getInfo()/1000).strftime("%Y-%m-%d")
print('Date-First Image: ',dateString)

# If working with S-2, then identify whether the sensor is Sentinel-2A or Sentinel-2B, and rename the variable 'mission'
if 'Sentinel2' == mission:
    mission = str(firstImage.getInfo()['properties']['SPACECRAFT_NAME'])
print('Mission: ', mission)

### Atmospheric Correction
Available bands available for atmospheric correction for each sensor, according to the [Py6S module](https://github.com/robintw/Py6S/blob/master/Py6S/Params/wavelength.py): <br/>
* **Sentinel2:** ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12'],<br/>
* **Landsat8:** ['B1','B2','B3','B4','B5','B6','B7','B8','B9'],<br/>
* **Landsat7:** ['B1','B2','B3','B4','B5','B7'],<br/>
* **Landsat5:** ['B1','B2','B3','B4','B5','B7'],<br/>
* **Landsat4:** ['B1','B2','B3','B4','B5','B7']<br/>

The respective cloud mask band will be preserved after the correction:<br/>
* **Sentinel2:** 'QA60',<br/>
* **Landsat sensors:** 'BQA'

**NOTE:** The function *positive* will convert any negative value to 0.0001 in all bands. For Sentinel-2, the bands B1,B2,B3,B4 are more susceptible to present negative values in very dark/coastal areas. I have compared those areas using Sentinel-2 L2A images and it seems they do the same: dark areas showing default minimum valid pixel values of 0.0001. B8 might show negative reflectances on coastal and oceanic areas, but not on cloudy pixels, so it does not affect cloud masking procedures. In my case, bands B8, B11, B12 are only used for masking clouds. If you need to use these bands for other purposes just check that the areas of negative values are not large, otherwise I would not recommend to use them (it is up to you). The negative reflectances might be due to overestimations of aerosols at sea-level in coastal areas and suspendend particles in water. Landsat sensors may show a similar behaviour. Band B10 should not be provided as a surface reflectance output, because it does not provide information on the surface but on the cirrus clouds [[Main-Knorn et al. 2017]](https://www.researchgate.net/publication/320231869_Sen2Cor_for_Sentinel-2). 

In [ ]:
## Default bands of interest:
if 'Sentinel' in mission:
    bands = ['B1','B2','B3','B4','B5','B8','B11','B12'] #Sentinel-2
elif 'Landsat8' in mission:
    bands = ['B1','B2','B3','B4','B5','B6','B7'] #Landsat-8
else:
    bands = ['B1','B2','B3','B4','B5','B7'] #Landsat-7/5

In [ ]:
## Function
## Surface reflectance (You can add more bands as desired, according to the above info)
def getBOA(coll, miss):
    List = coll.toList(coll.size())
    Size = List.size().getInfo()
    boaColl = ee.ImageCollection([])
    
    for i in range(Size):
        get = imageID[i]
        
        if 'Sentinel' in miss:
            miss = 'Sentinel2'
        
        img = ee.Image(mn.eeCollection(miss) + '/'+ get)
        imgInfo= ee.Image(img)        
        print('Processing Image '+str(i+1)+':', img.getInfo()['properties']['system:index'])
        
        ## Extract cloud band and thermal band for Landsat
        qa = []
        if 'Sentinel' in mission:
            qa = img.select('QA60')#For Sentinel
        elif 'Landsat8' in mission:
            qa = img.select('BQA') #For Landsat-8
            thermal = img.select('B10')
        elif 'Landsat7' in mission:
            qa = img.select('BQA') #For Landsat7
            thermal = img.select('B6_VCID_1')
        else:
            qa = img.select('BQA') #For Landsat5
            thermal = img.select('B6')
        
        if 'Sentinel2' == miss:
            miss = str(img.getInfo()['properties']['SPACECRAFT_NAME'])
            print('Mission: ', miss)
        
        output = ee.Image()
        for i in range(len(bands)):
            ## Get BOA reflectance for the respective band.
            b = BOA(miss, img, bands[i])

            ## Function to convert negative reflectances to 0.0001
            def positive(band):
                    ## If there is masked areas, unmask them and assign a specific pixel value different from 0.0001.
                    ## Sometimes Sentinel-2 tiles present cut off corners.
                    unmasked = band.unmask(9999)

                    ## Take all the positive pixel values and assing 0.0001 values to all negative ones.
                    b = unmasked.gt(0)
                    b_mask = unmasked.mask(b)
                    b_unmasked = b_mask.unmask(0.0001)

                    ## Re-mask the areas with 9999 values
                    remask = b_unmasked.neq(9999)

                    return ee.Image(b_unmasked).mask(remask)

            ## Bands to convert to positive.
            bPos = positive(b)

            ## Getting all the bands together
            output = output.addBands(bPos)
            
        ## Add QA band
        output = output.addBands(qa)
        
        ## Add thermal band if this is a Landsat image
        if 'Landsat' in mission:
            output = output.addBands(thermal)
        
        ## Copy properties from the original image
        output = output.set(img.toDictionary(img.propertyNames()))
            
        #print('Processed Image '+str(i)+':', output.getInfo()['properties']['system:index'])
        print('Done!')
        
        boaColl = boaColl.merge(ee.ImageCollection(output))
    
    return boaColl

Run atmospheric correction (for Sentinel, it takes ~20s per image (8 bands)).

In [ ]:
%%time
boaCollection = getBOA(collection, mission)

### Display results
Shows an image as example.

In [ ]:
from IPython.display import display, Image

boaImage = boaCollection.first()
region = boaImage.geometry().buffer(5000).bounds().getInfo()['coordinates']

# RGB Bands
channels = []
if 'Sentinel' in mission or 'Landsat8' == mission:
    channels = ['B4','B3','B2'] #For Sentinel & Landsat8
else:
    channels = ['B3','B2','B1'] #For Landsat7-5-4

# Display images:
original = Image(url=mn.TOA(firstImage,mission).select(channels).getThumbUrl({
    'dimensions': '1000x1000',
    'min':0,
    'max':0.25
    }))

corrected = Image(url=boaImage.select(channels).getThumbUrl({
    'dimensions': '1000x1000',
    'min':0,
    'max':0.25,
    'gamma':1.5
    }))

display(original, corrected)

### Export to Asset

NOTE: 
* Be aware that each band will be resampled at the scale used to export the image. This will impact the size of the exported file.
* A Sentinel-2 image with 8 bands at 10m res each can occupy ~1.5 gb.
* A Sentinel-2 image with 8 bands can take ~4-8 min to ingest.

In [ ]:
## Run the export

toaList = collection.toList(collection.size())
boaList = boaCollection.toList(boaCollection.size())
boaSize = boaList.size().getInfo()
print('Wait for submission')

for i in range(boaSize):
    ## Copy properties from the original image
    toa = ee.Image(toaList.get(i))
    image = ee.Image(boaList.get(i))
    #image = boa.set(toa.toDictionary(toa.propertyNames()))
    
    ## Set the scale properly
    scale = []
    sat = []
    tile = []
    if 'Sentinel' in mission:
        sat = 'Sentinel'
        scale = 10 #For Sentinel
        tile = (image.getInfo()['properties']['MGRS_TILE'])
    else:
        sat = 'Landsat'
        scale = 30 #For Landsat 
        tile = (image.getInfo()['properties']['WRS_PATH'])+(image.getInfo()['properties']['WRS_ROW'])
    
    ## Get properties from each image
    #imgID = image.getInfo()['properties']['system:index'] #This is returning a weird system:index
    imgID = toa.getInfo()['properties']['system:index'] #The solution is copying the id from the original collection.
    #print(imgID)
    date = datetime.datetime.utcfromtimestamp(image.get('system:time_start').getInfo()/1000).strftime("%Y-%m-%d")
    tileID = tile
    
    ## Set some properties for export
    output = image.set({'satellite': mission,
                   'tile_id': tileID,
                   'file_id': imgID,                                               
                   'date': date,
                   'generator': 'Lizcano-Sandoval',
                        })

    ## Define YOUR assetID. (This do not create folders, you need to create them manually)
    assetID = 'users/lizcanosandoval/BOA/'+sat+'/'+'FL_19/' ##This goes to an ImageCollection folder
    fileName = tileID+'_'+imgID+'_BOA'
    path = assetID + fileName

    ## Batch Export to Assets
    ee.batch.Export.image.toAsset(\
        image = ee.Image(output),                                                    
        description = 'S2_BOA_'+imgID,
        assetId = path,
        region = image.geometry().buffer(10),                                      
        maxPixels = 1e9,
        scale = scale).start()
    print('Image '+str(i+1)+': '+imgID+' submitted...')
print('All images submitted!')